<a href="https://colab.research.google.com/github/aman3013/Fine-tuning-Amharic-NER/blob/Task-5/Model_Interpretability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap lime

In [2]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

# Check the contents of the specified directory
model_path = '/content/drive/My Drive/fine_tuned_model'
print("Contents of the fine_tuned_model directory:")
print(os.listdir(model_path))


Contents of the fine_tuned_model directory:
['config.json', 'model.safetensors']


In [6]:
from transformers import AutoTokenizer

# Assuming you have used a pretrained model that you fine-tuned
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")  # or your original model

# Save the tokenizer files to your fine-tuned model directory
tokenizer.save_pretrained('/content/drive/My Drive/fine_tuned_model')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


('/content/drive/My Drive/fine_tuned_model/tokenizer_config.json',
 '/content/drive/My Drive/fine_tuned_model/special_tokens_map.json',
 '/content/drive/My Drive/fine_tuned_model/vocab.txt',
 '/content/drive/My Drive/fine_tuned_model/added_tokens.json',
 '/content/drive/My Drive/fine_tuned_model/tokenizer.json')

In [7]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Load the model and tokenizer from your Google Drive path
model_name = '/content/drive/My Drive/fine_tuned_model'  # Adjust this path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

In [10]:
# Load the .conll file into a list
data_path = '/content/drive/My Drive/merged_file.conll'  # Adjust this path

# Initialize lists to hold words and labels
words = []
labels = []

# Read the file line by line
with open(data_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()  # Remove leading and trailing whitespace
        if line:  # Check if the line is not empty
            parts = line.split()
            if len(parts) == 2:  # Ensure there are exactly two parts (word and label)
                words.append(parts[0])
                labels.append(parts[1])
            else:
                print(f"Skipping malformed line: {line}")  # Print or log malformed lines

# Create a new DataFrame for words and labels
ner_data = pd.DataFrame({'word': words, 'label': labels})

# Display the DataFrame
print(ner_data.head())
print(f"Total entries: {len(ner_data)}")



   word label
0   ይህን     O
1   መፍጫ     O
2  ከሁሉም     O
3  የተሻለ     O
4    ሆኖ     O
Total entries: 130348


In [13]:
# Randomly sample 5 rows from the DataFrame
sample_size = 20
sample_texts = ner_data.sample(n=sample_size, random_state=42)  # Set random_state for reproducibility

# Display the sampled texts
print(sample_texts)


              word  label
20160          ምርጥ      O
43275         ጎላጎል      O
5112           ዉስጥ      O
54618           ጊቢ      O
74357           ሰፊ      O
126989        የማይዝ      O
87820           ሱቅ  I-LOC
25029           10      O
53665   0911236983      O
67305         ያለውን      O
98360         ቅድስት  B-LOC
113114        አድራሻ  B-LOC
106448         አራት      O
59830          ያሉት      O
65619          ከሳሚ  I-LOC
55019       የሚገኝበት  I-LOC
23768          የሆኑ      O
75523        ዋናፅፈት  I-LOC
58286        በስተግራ  I-LOC
85026           ሞል  I-LOC


In [16]:
# Extract the 'word' column
word_column = ner_data['word']

# If you want to convert it to a list
word_list = word_column.tolist()

# Display the first few words
print(word_list[:10])  # Adjust the index to see more or fewer items


['ይህን', 'መፍጫ', 'ከሁሉም', 'የተሻለ', 'ሆኖ', 'አግኝተነዋል', '1', 'አስተማማኝ', 'የሆነ', 'ዕቃ']


In [20]:
import shap

# Initialize the SHAP explainer with your model and specify the tokenizer as the masker
masker = shap.maskers.Text(tokenizer)  # Use the tokenizer as the masker
explainer = shap.Explainer(model, masker)


In [ ]:
import pandas as pd
import shap
from transformers import AutoTokenizer

# Load the tokenizer from your fine-tuned model path
tokenizer_path = '/content/drive/My Drive/fine_tuned_model'  # Adjust this path
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Load your DataFrame (assuming ner_data is already defined)
# ner_data = pd.read_csv('your_data.csv')  # Load your data if not done yet

# Extract the 'word' column
word_column = ner_data['word']
word_list = word_column.tolist()

# Initialize the SHAP explainer with your model and specify the tokenizer as the masker
masker = shap.maskers.Text(tokenizer)  # Use the tokenizer as the masker
explainer = shap.Explainer(model, masker)

# Tokenize the input words for SHAP
tokenized_inputs = tokenizer(word_list, padding=True, truncation=True, return_tensors="pt")

# Generate SHAP values for the tokenized inputs
shap_values = explainer(tokenized_inputs)

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, word_list)


In [ ]:
import pandas as pd
import shap
from transformers import AutoTokenizer

# Load the tokenizer from your fine-tuned model path
tokenizer_path = '/content/drive/My Drive/fine_tuned_model'  # Adjust this path
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Load your DataFrame (assuming ner_data is already defined)
# ner_data = pd.read_csv('your_data.csv')  # Load your data if not done yet

# Extract the 'word' and 'label' columns
word_column = ner_data['word']
label_column = ner_data['label']

# Convert to lists
word_list = word_column.tolist()
label_list = label_column.tolist()  # If you need it later for analysis

# Tokenize the word list and convert to input IDs
tokenized_inputs = tokenizer(word_list, padding=True, truncation=True, return_tensors="pt")  # Use return_tensors to get PyTorch tensors

# Initialize the SHAP explainer with your model and specify the tokenizer as the masker
masker = shap.maskers.Text(tokenizer)  # Specify the tokenizer
explainer = shap.Explainer(model, masker)

# Generate SHAP values using the tokenized inputs
# We pass the input_ids along with the attention_mask
shap_values = explainer(tokenized_inputs['input_ids']

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, tokenized_inputs['input_ids'])





In [ ]:
from lime.lime_text import LimeTextExplainer

# Create a LIME explainer
lime_explainer = LimeTextExplainer(class_names=model.config.id2label.values())

# Function to predict the class for LIME
def lime_predict(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()
    return probabilities

# Explain a specific sample
i = 0  # Change this index to analyze different samples
lime_explanation = lime_explainer.explain_instance(
    sample_texts[i],
    lime_predict,
    num_features=10
)


In [ ]:
# SHAP summary plot
shap.summary_plot(shap_values, feature_names=tokenizer.convert_ids_to_tokens(tokenizer(sample_texts[0])['input_ids']))


In [ ]:
# LIME visualization
lime_explanation.show_in_notebook()
